In [19]:
# coding: utf-8
import os
import numpy as np
import sys
import pickle
import warnings
import cv2
import matplotlib.pyplot as plt
from tqdm import tqdm
warnings.filterwarnings('ignore')

# Dataset.pyの格納先を環境変数に追加する
sys.path.append("../src/utils")
sys.path.append("../src")
from Dataset import Dataset

# 転移学習用
from keras import optimizers
from keras.utils.np_utils import to_categorical
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, Dropout, Flatten, Input, Activation, BatchNormalization
from keras.models import Model, Sequential
from keras.callbacks import CSVLogger
from keras.callbacks import ModelCheckpoint

In [20]:
# モデルの定義
input_tensor = Input(shape=(100, 100, 3))
vgg16 = VGG16(include_top=False, weights='imagenet', input_tensor=input_tensor)

top_model = Sequential()
top_model.add(Flatten(input_shape=vgg16.output_shape[1:]))
top_model.add(Dense(128))
top_model.add(Activation('relu'))
top_model.add(BatchNormalization())
top_model.add(Dropout(0.5))
top_model.add(Dense(128))
top_model.add(Activation('relu'))
top_model.add(BatchNormalization())
top_model.add(Dropout(0.5))
top_model.add(Dense(128))
top_model.add(Activation('relu'))
top_model.add(BatchNormalization())
top_model.add(Dropout(0.5))
top_model.add(Dense(2))
top_model.add(Activation('softmax'))

model = Model(inputs=vgg16.input, outputs=top_model(vgg16.output))

# 15層目までは学習を行わない
for layer in model.layers[:15]:
    layer.trainable = False

# モデルをコンパイル
model.compile(loss='binary_crossentropy',
              optimizer=optimizers.Adam(),
              metrics=['accuracy'])

In [21]:
# データセットを定義
dataset = Dataset()
# pickleの格納先
pickle_path = "../data/dataset.pkl"
# pickleからデータセットを読み込む
dataset.read_pickle(pickle_path)

# 学習データとテストデータの割合
ratio = 0.2
# 学習データとテストデータに分ける
dataset.split_data(ratio)

# one-hot-vectorに変換
dataset.y_train = to_categorical(dataset.y_train)
dataset.y_test = to_categorical(dataset.y_test)

In [22]:
print(1 - dataset.y_test.argmax(axis=1).sum()/dataset.y_test.shape[0])
print(dataset.y_test.argmax(axis=1).sum()/dataset.y_test.shape[0])

0.7057926829268293
0.2942073170731707


In [ ]:
model_name = "VGG16_finetuning_Adam_1000"
model_path = "../models/{}".format(model_name)

if not os.path.exists(model_path):
    os.mkdir(model_path)

cl = CSVLogger('{}/{}.csv'.format(model_path, model_name))

cp = ModelCheckpoint('../models/VGG16_finetuning_Adam_1000/VGG16_{epoch:02d}.h5',
                     monitor="val_acc", verbose=1,
                     save_best_only=False, save_weights_only=True)

model.save("{}/VGG16.h5".format(model_path))

history = model.fit(
              dataset.X_train, dataset.y_train,
              batch_size=64, epochs=1000,
              validation_data=(dataset.X_test, dataset.y_test),
              callbacks=[cp, cl])

Train on 2624 samples, validate on 656 samples
Epoch 1/1000
2624/2624 [==============================] - 7s 3ms/step - loss: 1.0905 - accuracy: 0.5412 - val_loss: 2.8240 - val_accuracy: 0.7058

Epoch 00001: saving model to ../models/VGG16_finetuning_Adam_1000/VGG16_01.h5
Epoch 2/1000
2624/2624 [==============================] - 6s 2ms/step - loss: 0.9114 - accuracy: 0.5979 - val_loss: 1.0313 - val_accuracy: 0.7058

Epoch 00002: saving model to ../models/VGG16_finetuning_Adam_1000/VGG16_02.h5
Epoch 3/1000
2624/2624 [==============================] - 6s 2ms/step - loss: 0.8061 - accuracy: 0.6239 - val_loss: 0.7869 - val_accuracy: 0.7058

Epoch 00003: saving model to ../models/VGG16_finetuning_Adam_1000/VGG16_03.h5
Epoch 4/1000
2624/2624 [==============================] - 6s 2ms/step - loss: 0.7168 - accuracy: 0.6612 - val_loss: 0.5863 - val_accuracy: 0.6875

Epoch 00004: saving model to ../models/VGG16_finetuning_Adam_1000/VGG16_04.h5
Epoch 5/1000
2624/2624 [=============================

In [ ]:
predict_class = model.predict(dataset.X_test).argmax(axis=1)

In [ ]:
predict_class

In [6]:
num = 500
for i, (X_pre, y_pre, path_pre) in enumerate(zip(dataset.X_test, dataset.y_test, dataset.path_test)):
    if i == num:
        X_pre = X_pre.reshape((1, 100, 100, 3))
        print(model.predict(X_pre))
        img = cv2.imread(".{}".format(path_pre))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.imshow(img)
        if y_pre.argmax() == model.predict(X_pre).argmax():
            print("正解！")
        else:
            print("不正解...")
        print("正解ラベル: ", end="")
        if y_pre[0]==1:
            print("タイプでない")
        elif y_pre[1]==1:
            print("タイプ!!!!")

[[9.9999928e-01 6.6150324e-07]]


error: OpenCV(4.1.2) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'
